<a href="https://colab.research.google.com/github/AraiKensuke/AIiRPS/blob/master/Launch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/AraiKensuke/AIiRPS.git
!cd AIiRPS

###  First, clone the pyPG package from Github.  This will only last for this runtime session.  Do this every time you re-open notebook.
!git clone https://github.com/AraiKensuke/pyPG.git
%cd /content/pyPG/

###  pyPG is written in C++/python, and uses the C library GNU Scientific Library.  
!sudo apt-get install libgsl-dev   

###  setup and build pyPG
!python setup.py build_ext --inplace

%cd /content

In [7]:
import AIiRPS

In [6]:
import pyPG